# Задание на практику

1) Найти данные для задачи классификации или для задачи регрессии.
2) Реализовать баггинг.
3) Реализовать бустинг на тех же данных, что использовались для баггинга.
4) Сравнить результаты работы алгоритмов (время работы и качество моделей). Сделать выводы.
5) Оформить отчет о проделанной работе.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import GridSearchCV

from sklearn.datasets import fetch_openml
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split

In [ ]:
data_list = [{'data':pd.read_csv('/content/WorldHappiness_Corruption_2015_2020.csv'), 'target':'happiness_score'}, # regression
             {'data':pd.read_csv('/content/Car_Insurance_Claim.csv').iloc[:4000, :], 'target':'OUTCOME'}] # classification

In [ ]:
for data in data_list:
  data['target'] = data['data'][[data['target']]]

In [ ]:
data_list[0]['data']

,Country,happiness_score,gdp_per_capita,family,health,freedom,generosity,government_trust,dystopia_residual,continent,Year,social_support,cpi_score
0,Norway,7.5370,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027,Europe,2015,0.000000,88
1,Denmark,7.5220,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707,Europe,2015,0.000000,91
2,Iceland,7.5040,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715,Europe,2015,0.000000,79
3,Switzerland,7.4940,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716,Europe,2015,0.000000,86
4,Finland,7.4690,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182,Europe,2015,0.000000,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,Botswana,3.4789,0.997549,0.000000,0.494102,0.509089,0.033407,0.101786,0.257241,Africa,2020,1.085695,60
788,Tanzania,3.4762,0.457163,0.000000,0.442678,0.509343,0.271541,0.203881,0.718963,Africa,2020,0.872675,38
789,Rwanda,3.3123,0.343243,0.000000,0.572383,0.604088,0.235705,0.485542,0.548445,Africa,2020,0.522876,54
790,Zimbabwe,3.2992,0.425564,0.000000,0.375038,0.377405,0.151349,0.080929,0.841031,Africa,2020,1.047835,24


In [ ]:
data_list[1]['data']

,ID,AGE,GENDER,RACE,DRIVING_EXPERIENCE,EDUCATION,INCOME,CREDIT_SCORE,VEHICLE_OWNERSHIP,VEHICLE_YEAR,MARRIED,CHILDREN,POSTAL_CODE,ANNUAL_MILEAGE,VEHICLE_TYPE,SPEEDING_VIOLATIONS,DUIS,PAST_ACCIDENTS,OUTCOME
0,569520,65+,female,majority,0-9y,high school,upper class,0.629027,1.0,after 2015,0.0,1.0,10238,12000.0,sedan,0,0,0,0.0
1,750365,16-25,male,majority,0-9y,none,poverty,0.357757,0.0,before 2015,0.0,0.0,10238,16000.0,sedan,0,0,0,1.0
2,199901,16-25,female,majority,0-9y,high school,working class,0.493146,1.0,before 2015,0.0,0.0,10238,11000.0,sedan,0,0,0,0.0
3,478866,16-25,male,majority,0-9y,university,working class,0.206013,1.0,before 2015,0.0,1.0,32765,11000.0,sedan,0,0,0,0.0
4,731664,26-39,male,majority,10-19y,none,working class,0.388366,1.0,before 2015,0.0,0.0,32765,12000.0,sedan,2,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,557897,40-64,female,majority,0-9y,high school,middle class,0.543401,1.0,before 2015,1.0,1.0,32765,11000.0,sedan,0,0,0,1.0
3996,253559,16-25,female,minority,0-9y,none,poverty,0.369953,1.0,before 2015,0.0,0.0,10238,16000.0,sedan,0,0,0,0.0
3997,577935,16-25,female,majority,0-9y,high school,poverty,NaN,1.0,before 2015,0.0,1.0,32765,9000.0,sedan,0,0,0,1.0
3998,416198,65+,female,majority,30y+,high school,upper class,0.560581,0.0,before 2015,1.0,1.0,32765,NaN,sedan,4,2,2,0.0


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def cat_cols(data):
  return list(data['data'].select_dtypes(include=["object"]).columns)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

def train_models(models, data, metric):
    X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=0.2)
    # Random forest
    transform = ColumnTransformer([('onehot', OneHotEncoder(drop='if_binary'), cat_cols(data))])
    print("Random forest:")
    bagging, boosting = None, None
    bagging = %time make_pipeline(transform, StandardScaler(with_mean=False), models[0]).fit(X_train, y_train.values.ravel())
    # print("best_params_:", bagging[-1].best_params_)
    # Gradient boosting
    print("Boosting:")
    boosting = %time models[1].fit(X_train, y_train, cat_features=cat_cols(data), verbose=False)
    return metric(y_test, bagging.predict(X_test)), metric(y_test, boosting.predict(X_test))

In [ ]:
# param_grid = {
#     'max_depth': np.linspace(3, 10, 5, dtype=int),
#     'min_samples_leaf': np.linspace(2, 20, 4, dtype=int),
#     'min_samples_split': np.linspace(2, 20, 4, dtype=int),
# }
# models = [
#     (GridSearchCV(RandomForestRegressor(), param_grid=param_grid, cv=8),
#      CatBoostRegressor()),
#     (GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv=8),
#      CatBoostClassifier())
# ]

models = [
    (RandomForestRegressor(max_depth=9, 
                           min_samples_leaf=2, 
                           min_samples_split=8, 
                           criterion='squared_error'),
     CatBoostRegressor()),
    (RandomForestClassifier(max_depth=9, 
                            min_samples_leaf=10, 
                            min_samples_split=4,
                            criterion='entropy'),
     CatBoostClassifier())
]

In [ ]:
from sklearn.metrics import recall_score, r2_score

results = []
for data, _models, task, metric in zip(data_list, models, 
                                       ['Regression:', 'Classification:'],
                                       [r2_score, recall_score]):
  print(task, end='\n\n')
  results.append(train_models(_models, data, metric))
results

Regression:

Random forest:
CPU times: user 192 ms, sys: 2.02 ms, total: 194 ms
Wall time: 201 ms
Boosting:
CPU times: user 7.21 s, sys: 552 ms, total: 7.76 s
Wall time: 4.16 s
Classification:

Random forest:
CPU times: user 392 ms, sys: 968 µs, total: 393 ms
Wall time: 395 ms
Boosting:
CPU times: user 11.4 s, sys: 865 ms, total: 12.2 s
Wall time: 6.76 s


[(0.7604052140318496, 0.9989103937975068), (0.6404494382022472, 1.0)]

Вывод при кросс-валидации

```
Regression:

Random forest:
CPU times: user 1min 40s, sys: 708 ms, total: 1min 41s
Wall time: 1min 44s
best_params_: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 8}
Boosting:
CPU times: user 7 s, sys: 484 ms, total: 7.49 s
Wall time: 4.29 s
Classification:

Random forest:
CPU times: user 3min 2s, sys: 836 ms, total: 3min 3s
Wall time: 3min 3s
best_params_: {'max_depth': 10, 'min_samples_leaf': 14, 'min_samples_split': 2}
Boosting:
CPU times: user 11 s, sys: 870 ms, total: 11.8 s
Wall time: 6.52 s
[[(0.6971858500906216, 0.9985388023268945)], [(0.6782945736434108, 1.0)]]
```

